
# 商品マスタを使って、Attachmentのタグを自動生成させるために、
# 商品説明の要約を行う



タイムアウト切れ対応

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

## ライブラリやデータの準備

永続化のためにGoogleDriveのマイドライブを接続しておく

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


デフォルトGドライブパスを指定

In [ ]:
fldGdrive = '/content/drive/'
fldGdriveMyDrive = fldGdrive + 'MyDrive/'
fldGdriveMyDriveAttachment = fldGdriveMyDrive + 'Attachment/'
fldGdriveMyDriveAttachmentWorkT5 = fldGdriveMyDriveAttachment + 'work/t5/'

依存ライブラリのインストール

In [ ]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2 torchaudio==0.7.2
!pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.2/814.2 KB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 54.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## 各種ディレクトリ作成

* data: 学習用データセット格納用
* model: 学習済みモデル格納用

In [ ]:
!mkdir -p {fldGdriveMyDriveAttachmentWorkT5}data {fldGdriveMyDriveAttachmentWorkT5}data/learning {fldGdriveMyDriveAttachmentWorkT5}model
%ls {fldGdriveMyDriveAttachmentWorkT5}

data/  model/  old/


In [ ]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 転移学習済みモデル
MODEL_DIR = fldGdriveMyDriveAttachmentWorkT5 + "model"

事前学習済みモデル<br>
転移学習させない場合に利用。

In [ ]:
#MODEL_DIR = "sonoisa/t5-base-japanese-title-generation"

クライアントから連携された商品マスタから、以下の形式で連携(アップロード)しておく

* ファイルフォーマット： <br>
1行目は、ヘッダーであること<br>
[行番号], [商品ID] | [商品名] | [商品説明]  
:
---
* 配置ディレクトリ
  >input(指定)ディレクトリ
  >>txt
  >>>products
  >>>>description_[行番号]_[商品ID].txt<br>
  :
---

* 配置したディレクトリごｔtar.gzで圧縮



文字列の正規化の定義<br>
表記揺れを減らします。今回はneologdの正規化処理を一部改変したものを利用します。 処理の詳細はリンク先を参照してください。<br>
https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変

In [ ]:
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

Dataが作業フォルダへ<br>
取り込むtar.gzファイルがあるかをチェック

In [ ]:
%cd /content/drive/MyDrive/Attachment/work/t5/data/
%ls

/content/drive/MyDrive/Attachment/work/t5/data
aoscs_20230113_prodcuts_descriptions_1_1000.tar.gz  learning/  lightning_logs/


tar.gzから情報を抽出

In [ ]:
import tarfile
import re

target_genres = ["products"]

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def read_title_body(file):
    next(file)
    title = next(file).decode("utf-8").strip()
    title = normalize_text(remove_brackets(title))
    body = normalize_text(" ".join([line.decode("utf-8").strip() for line in file.readlines()]))
    return title, body
    
def read_id_name_desc(file):
    line = next(file).decode("utf-8").strip()
    line = normalize_text(remove_brackets(line))
    row = line.split('|')
    id = row[0].split(",")[1]
    name = row[1]
    desc = row[2]
    return id, name, desc

genre_files_list = [[] for genre in target_genres]

all_data = []
                
def read_tarfile_(tar):
  with tarfile.open(tar) as archive_file:
      for archive_item in archive_file:
          for i, genre in enumerate(target_genres):
              if genre in archive_item.name and archive_item.name.endswith(".txt"):
                  genre_files_list[i].append(archive_item.name)
      for i, genre_files in enumerate(genre_files_list):
          for name in genre_files:
              file = archive_file.extractfile(name)
              id, name, desc = read_id_name_desc(file)
              name = normalize_text(name)
              desc = normalize_text(desc)
              print(id, name)

              if len(id) > 0 and len(name) > 0:
                  all_data.append({
                      "id": id,
                      "name": name,
                      "descrption": desc
                      })



In [ ]:
tar = "aoscs_20230113_prodcuts_descriptions_1_1000.tar.gz"
read_tarfile_(tar)
print(all_data)

70273 ワイドカラースタイリッシュワイシャツ【controlα】
68721 ワイドカラースタイリッシュワイシャツ【non irontech】
70515 ワイドカラースタイリッシュワイシャツ【non ironmax】
72336 ワイドカラースタイリッシュワイシャツ
70308 モバイルバックパック【撥水加工】
69370 クルーネックロングtシャツ
71775 スタンダードスーツ【ウォッシャブル】【キング】
67905 ワイドカラースタンダードワイシャツ【キング&トール】
70352 プレミアムレギュラータイ【西陣織】
69617 ワイドカラースタンダードワイシャツ【non irontech】
68769 オンラインストア・一部店舗取り扱いクルーネックロングスリーブ【controlα】
70287 レギュラータイ【recommend for freshmen】
69695 ワイドカラースタイリッシュワイシャツ【tiotio premium】
72204 ボタンダウンスタンダードワイシャツ【キング&トール】【controlα】
72359 レギュラーカラースタンダードワイシャツ
72216 レギュラータイ【oeko-tex】
69850 【男女兼用】デニム調ジョガーパンツ【裾上げ済み】【セットアップ】
71772 スタイリッシュスーツ【スリーピース】
71349 バックパック【tiotio premium】
68035 ボタンダウンスタンダードワイシャツ【キング&トール】【non irontech】
67898 スタイリッシュスーツ【plastics smart】
70290 レギュラータイ【recommend for freshmen】
71100 レギュラータイ【oeko-tex】
70358 プレミアムレギュラータイ【西陣織】
71199 プレミアムレギュラータイ【made in italy】
71110 レギュラータイ【oeko-tex】
67911 ボタンダウンスタンダードワイシャツ【キング&トール】
68128 レギュラータイ【vintage&heritage】
71245 タイクリップ【baseball bat】
70320 ピンバックルメンズベルト
68722 ワイドカラースタイリッシュワイシャツ【non irontech】
69904 ワイド

データを分割<br>
データセットを90% : 5%: 5% の比率でtrain/dev/testに分割<br>
<br>


* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [ ]:
%ls {fldGdriveMyDriveAttachmentWorkT5}data/learning

dev.tsv  test.tsv  train.tsv


In [ ]:
#出力するパスを指定しておく
fldGdriveMyDriveAttachmentWorkT5DataLearning=fldGdriveMyDriveAttachmentWorkT5+'data/learning/'

In [ ]:
import random
from tqdm import tqdm

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    id = data["id"]
    name = data["name"]
    descrption = data["descrption"]

    assert len(id) > 0 and len(name) > 0
    return f"{id}\t{name}\t{descrption}\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.9, 0.05, 0.05

with open(f"{fldGdriveMyDriveAttachmentWorkT5DataLearning}train.tsv", "w", encoding="utf-8") as f_train, \
    open(f"{fldGdriveMyDriveAttachmentWorkT5DataLearning}dev.tsv", "w", encoding="utf-8") as f_dev, \
    open(f"{fldGdriveMyDriveAttachmentWorkT5DataLearning}test.tsv", "w", encoding="utf-8") as f_test:
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

1001it [00:00, 79660.34it/s]


作成されたデータを確認
形式: {ID} \t {Name} \t {Descroption}

In [ ]:
%cat {fldGdriveMyDriveAttachmentWorkT5DataLearning}test.tsv | head -3

70314	ピンバックルメンズベルト	【商品詳細】きめ細やかでしなやかな皮革を使用して作られたメンズベルトです。シンプルなバックルを取り付け、さりげなくウエストを飾ります。バックル部分は取り外し可能でベルト裏側にはサイズカット用の目盛りを備え、お好みのサイズにカットしていただけるように仕上げました。ビジネスシーンではベルトとシューズの色は同色でまとめると綺麗です。【grain leather】グレインレザーとは「本革」を意味しています。天然素材ならではの質感があり革本来の風合いが魅力です。【革製品注意点】天然素材のため、キズやシワが入っている場合がございます。また、革質・厚みによって染色の度合いが変わり、色の違いが生じる場合がございます。【商品に関するご注意】■ブラウザやお使いのモニター環境、室内外等の撮影時の環境下での光加減により、実際の商品と掲載画像の色味が異なる場合がございます。■平置き・メジャーでの採寸の為、素材や仕様等により実際の商品とサイズに若干の誤差が生じる場合がございます。予めご了承ください。
71798	ボタンダウンスタイリッシュワイシャツ	【商品詳細】伸縮性に優れたストレッチ素材を使用、快適な着用感を与えます。スッキリしたスタイリングのワイシャツはスタイリッシュな細身のスーツやジャケットに合わせてもジャケット内でのダブつきを抑え、動きを阻害することなくスッキリと着用いただけます。【仕様・機能】■super easy iron形態安定性に優れ洗濯後のアイロン掛けも簡単です。■ストレッチ心地よい伸縮性により、快適な着心地をサポート。【シルエット】《細め(スリム)》(当社比)■ゆとり感には個人差があります。サイズ表を確認の上、ご購入の目安としてご利用ください。【商品に関するご注意】■ブラウザやお使いのモニター環境、室内外等の撮影時の環境下での光加減により、実際の商品と掲載画像の色味が異なる場合がございます。■平置き・メジャーでの採寸の為、素材や仕様等により実際の商品とサイズ表に若干の誤差が生じる場合がございます。予めご了承ください。
70092	【男女兼用】フルジップジャージジャケット【セットアップ】	【商品詳細】《家でも外でも心地いいワンマイルウエア》裏面がメッシュ調のジャージ素材を使用した、使いやすいベーシックなリラクシングウエアです。s〜3lまで

# 推論に必要なクラス等の定義
推論にはPyTorch/Transformersを利用する。

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

GPU利用有無

In [ ]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir= fldGdriveMyDriveAttachmentWorkT5DataLearning,  # データセットのディレクトリ
    model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    # max_input_length=512,
    # max_target_length=64,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)

## TSVデータセットクラス
TSV形式のファイルをデータセットとして読み込み<br>
形式: {ID} \t {Name} \t {Descroption}

In [ ]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)

        print(self.file_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, name, description, id):
        #入出力形式に変換する。
        input = f"{description}"
        target = f"{name}"
        return input, target
  
    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 3
                assert len(line[0]) > 0
                assert len(line[1]) > 0
                assert len(line[2]) > 0

                id = line[0]
                name = line[1]
                description = line[2]

                input, target = self._make_record(name, description, id)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)

試しにテストデータ（test.tsv）を読み込み、トークナイズ結果を確認

In [ ]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)

# テストデータセットの読み込み
train_file = fldGdriveMyDriveAttachmentWorkT5DataLearning+'train.tsv'
#print(args_dict["data_dir"])
#max_input_length = args_dict["max_input_length"]
#max_target_length = args_dict["max_target_length"]
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], train_file, 
                           input_max_len=512, 
                           target_max_len=64)

/content/drive/MyDrive/Attachment/work/t5/data/learning/train.tsv


テストデータの1レコード目をチェック


In [ ]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
【handcrafted and sustainable】オーセンティックなスタイルでありながら、足元の存在感を際立たせ、さらに地球環境やサスティナビリティを考えたドレスシューズです。キメ細かく上品なアッパーのレザーは、国際機関のlwgの承認を受けたタンナーで生産されたものを使用しています。美しいフォルムながら、日本人に合わせて幅広いスタイルに設計されたhiltonオリジナルモデルです。【lwg認証レザー】国際機関であるlwg(レザーワーキンググループ)の承認を受けたタンナーで生産されたレザーを使用した、環境に配慮されたサスティナブル商品です。【革製品注意点】■天然素材のため、キズやシワが入っている場合がございます。また、革質・厚みによって染色の度合いが変わり、色の違いが生じる場合がございます。■この製品は素材本来の風合いを生かすため、必要以上の色止めをしておらず、色落ちは避けられません。白・淡色系の衣類などとの併用はご注意ください。【商品に関するご注意】■ブラウザやお使いのモニター環境、室内外等の撮影時の環境下での光加減により、実際の商品と掲載画像の色味が異なる場合がございます。■平置き・メジャーでの採寸の為、素材や仕様等により実際の商品とサイズに若干の誤差が生じる場合がございます。予めご了承ください。</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

## ((学習済みモデルの読み込み))

In [ ]:
#
#import torch
#from torch.utils.data import Dataset, DataLoader
#from transformers import T5ForConditionalGeneration, T5Tokenizer
#
## トークナイザー（SentencePiece）
#tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)
#
## 学習済みモデル
#trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)
#
## GPUの利用有無
#USE_GPU = torch.cuda.is_available()
#if USE_GPU:
#    trained_model.cuda()

## 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.hparams.eval_batch_size, 
                          num_workers=4)

## 転移学習を実行


GPUのOut Of Memoryエラーが発生することがあります。
その場合、次の順にハイパーパラメータの調整を試してみるとエラーを解消できる場合があります。

1. 訓練時のバッチサイズ train_batch_size を小さくする（例えば4）。  
小さくしすぎると精度が悪化することがあります。
2. 入力文の最大トークン数 max_input_length や出力文の最大トークン数 max_target_length を小さくする（例えば、入力を256や出力を32にする）。  
入力文の最大トークン数を小さくすると一般に精度が落ち、出力文の最大トークン数を小さくすると生成できる文章の長さが短くなります。

In [ ]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  512,  # 入力文の最大トークン数
    "max_target_length": 64,  # 出力文の最大トークン数
    "train_batch_size":  8,  # 訓練時のバッチサイズ
    "eval_batch_size":   8,  # テスト時のバッチサイズ
    "num_train_epochs":  8,  # 訓練するエポック数
    })
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
)

In [ ]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores


/content/drive/MyDrive/Attachment/work/t5/data/learning/train.tsv
/content/drive/MyDrive/Attachment/work/t5/data/learning/dev.tsv



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)
INFO:lightning:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]